<a href="https://colab.research.google.com/github/afungue/DataExtraction/blob/main/f1-consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenF1 https://openf1.org/
OpenF1 es una API gratuita y de código abierto que proporciona datos en tiempo real e históricos de la Fórmula 1.

La API ofrece una gran cantidad de información, incluyendo tiempos por vuelta, telemetría de los autos, comunicaciones por radio, y más. Ya sea que estés buscando crear paneles interactivos, profundizar en el análisis de las carreras o incluso desarrollar objetos conectados que se iluminen cada vez que tu piloto favorito tome la delantera, OpenF1 lo hace posible.

Los datos se pueden acceder en formatos JSON o CSV, lo que la hace amigable tanto para desarrolladores como para personas sin experiencia técnica. Para comenzar rápidamente, puedes acceder a la API a través de tu navegador web. Se proporciona una URL de ejemplo para cada método como referencia.

In [ ]:
!pip install requests


# Sesiones
Las sesiones disponibles van desde la 0 hasta la 202 (Austin Texas 2024)

In [ ]:
from urllib.request import urlopen, Request
import json
import os

# Define la URL de la API para obtener todas las sesiones
url = 'https://api.openf1.org/v1/sessions'
api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

# Configura la solicitud con la clave API en los encabezados
request = Request(url)
request.add_header('Authorization', f'Bearer {api_key}')

try:
    # Realiza la solicitud a la API
    response = urlopen(request)
    data = response.read().decode('utf-8')

    # Cargar la respuesta como JSON
    data_sessions = json.loads(data)

    # Ordenar las sesiones por año (descendente) y date_start (descendente)
    sorted_sessions = sorted(data_sessions, key=lambda x: (x['year'], x['date_start']), reverse=True)

    # Imprimir la información de todas las sesiones en el formato solicitado
    last_year = None
    last_location = None

    for session in sorted_sessions:  # Iterar sobre las sesiones ya ordenadas
        if (session['year'], session['location']) != (last_year, last_location):
            # Solo imprime el año y la localidad una vez
            date_start = session['date_start']
            date_end = session['date_end']
            print(f"{session['year']}, {session['location']} (Inicio: {date_start}, Fin: {date_end})")
            last_year = session['year']
            last_location = session['location']

        # Indentación para las sesiones
        print(f"    {session['session_name']} (ID: {session['session_key']})")

except Exception as e:
    print("An error occurred:", str(e))


# Sesiones por año


In [ ]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# Función para obtener sesiones por año
def get_sessions_by_year(year):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Ordenar las sesiones por date_start de más nueva a más antigua
        data_sessions.sort(key=lambda x: x['date_start'], reverse=True)

        # Imprimir la información de las sesiones
        current_year = None
        current_location = None
        print(f"{'Año':<6} {'Localidad':<20} {'Nombre':<30} {'ID':<6} {'Fecha inicio':<25} {'Fecha fin':<25}")
        print("=" * 120)

        # Definir la zona horaria de Argentina (GMT-3)
        tz = pytz.timezone('America/Argentina/Buenos_Aires')

        for session in data_sessions:
            if session['year'] != current_year:
                current_year = session['year']
                print(f"{current_year:<6}")

            if session['location'] != current_location:
                current_location = session['location']
                print(f"  {current_location:<20}")

            # Convertir las fechas a tu zona horaria
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00"))
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00"))

            # Ajustar a la zona horaria GMT-3
            date_start = date_start.astimezone(tz)
            date_end = date_end.astimezone(tz)

            # Formatear las fechas
            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            # Imprimir nombre de sesión, ID y fechas con ajuste de espacio
            print(f"    {session['session_name']:<18} (ID: {session['session_key']:<5}) - Fecha inicio: {formatted_start:<24} - Fecha fin: {formatted_end:<24}")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_sessions_by_year(2024)  # Cambia el año según lo necesites


# Localidades por año

In [55]:
from urllib.request import urlopen, Request
import json
import os

# Función para obtener localidades por año
def get_locations_by_year(year):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Usar un conjunto para almacenar las localidades únicas
        locations = set()

        # Recorrer las sesiones y añadir las localidades al conjunto
        for session in data_sessions:
            locations.add(session['location'])

        # Imprimir las localidades
        print(f"Localidades donde se corrió en {year}:")
        for location in sorted(locations):
            print(location)

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_locations_by_year(2024)  # Cambia el año según lo necesites


Localidades donde se corrió en 2024:
Austin
Baku
Barcelona
Budapest
Imola
Jeddah
Marina Bay
Melbourne
Miami
Monaco
Montréal
Monza
Sakhir
Shanghai
Silverstone
Spa-Francorchamps
Spielberg
Suzuka
Zandvoort


# Sesiones por año y localidad

In [56]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# Función para obtener sesiones por año y localidad
def get_sessions_by_year_and_location(year, location):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Filtrar las sesiones por localidad
        filtered_sessions = [session for session in data_sessions if session['location'].lower() == location.lower()]

        # Ordenar las sesiones por date_start de más nueva a más antigua
        filtered_sessions.sort(key=lambda x: x['date_start'], reverse=True)

        # Imprimir la información de las sesiones
        current_year = None
        current_location = None
        print(f"{'Año':<6} {'Localidad':<20} {'Nombre':<30} {'ID':<6} {'Fecha inicio':<25} {'Fecha fin':<25}")
        print("=" * 120)

        # Definir la zona horaria de Argentina (GMT-3)
        tz = pytz.timezone('America/Argentina/Buenos_Aires')

        for session in filtered_sessions:
            if session['year'] != current_year:
                current_year = session['year']
                print(f"{current_year:<6}")

            if session['location'] != current_location:
                current_location = session['location']
                print(f"  {current_location:<20}")

            # Convertir las fechas a tu zona horaria
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00"))
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00"))

            # Ajustar a la zona horaria GMT-3
            date_start = date_start.astimezone(tz)
            date_end = date_end.astimezone(tz)

            # Formatear las fechas
            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            # Imprimir nombre de sesión, ID y fechas con ajuste de espacio
            print(f"    {session['session_name']:<18} (ID: {session['session_key']:<5}) - Fecha inicio: {formatted_start:<24} - Fecha fin: {formatted_end:<24}")

        # Si no hay sesiones filtradas, informar al usuario
        if not filtered_sessions:
            print(f"No se encontraron sesiones en {location} para el año {year}.")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_sessions_by_year_and_location(2024, 'Baku')  # Cambia el año y la localidad según lo necesites


Año    Localidad            Nombre                         ID     Fecha inicio              Fecha fin                
2024  
  Baku                
    Race               (ID: 9598 ) - Fecha inicio: 15 September 2024, 08:00 - Fecha fin: 15 September 2024, 10:00
    Qualifying         (ID: 9594 ) - Fecha inicio: 14 September 2024, 09:00 - Fecha fin: 14 September 2024, 10:00
    Practice 3         (ID: 9593 ) - Fecha inicio: 14 September 2024, 05:30 - Fecha fin: 14 September 2024, 06:30
    Practice 2         (ID: 9592 ) - Fecha inicio: 13 September 2024, 10:00 - Fecha fin: 13 September 2024, 11:00
    Practice 1         (ID: 9591 ) - Fecha inicio: 13 September 2024, 06:30 - Fecha fin: 13 September 2024, 07:30
